In [ ]:
import cv2
from ultralytics import YOLO

# Load your model onto GPU
model = YOLO('yolov8n.pt').to('cuda:0')

# Start streaming + tracking
stream = model.track(
    source=r'D:\self study\0 - FYP\Seatify\server\objectDetection\assets\video5.mp4',
    tracker='bytetrack.yaml',
    classes=[0, 56, 60],     # person, chair, dining table
    persist=True,
    show=False,              # we’ll show manually
    save=True,
    save_dir='my_outputs/',  # where annotated video is saved
    stream=True,             # frame‐by‐frame generator
    device='cuda:0'
)

# Define BGR colors for each label
colors = {
    'person':       (0,   0, 255),  # red
    'chair':        (0, 255,   0),  # green
    'dining table': (255, 0,   0)   # blue
}

for frame_idx, res in enumerate(stream):
    frame = res.orig_img.copy()  # get the raw frame as a NumPy array

    # If there are no detections, just display/skip
    if not res.boxes or len(res.boxes.xyxy) == 0:
        print(f"Frame {frame_idx}: no detections")
    else:
        print(f"Frame {frame_idx}:")
        # iterate in parallel through each detection
        xyxys     = res.boxes.xyxy.cpu().numpy()
        confs     = res.boxes.conf.cpu().numpy()
        cls_ids   = res.boxes.cls.cpu().numpy().astype(int)
        track_ids = res.boxes.id.cpu().numpy().astype(int)

        for (x1, y1, x2, y2), conf, cls_id, track_id in zip(xyxys, confs, cls_ids, track_ids):
            label = res.names[cls_id]              # e.g. "person", "chair", "dining table"
            color = colors.get(label, (255,255,255))  # white fallback

            # draw box & text
            cv2.rectangle(frame, (int(x1),int(y1)), (int(x2),int(y2)), color, 2)
            text = f"{label} {conf:.2f} ID:{track_id}"
            cv2.putText(frame, text, (int(x1), int(y1)-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
            print(f"  → {label:12s} conf={conf:.2f}  box=[{int(x1)},{int(y1)},{int(x2)},{int(y2)}]  track_id={track_id}")

    # show the custom‐colored frame
    cv2.imshow("Tracking (q to quit)", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
